In [1]:
#function ConnectButton(){
#    console.log('Working');
#    document.querySelector('#connect').click()
#}
#setInterval(ConnectButton, 60000);

In [2]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')

In [3]:
df.drop(['PhraseId', 'SentenceId'], axis=1, inplace=True)

In [4]:
df.shape

(156060, 2)

In [5]:
df_0 = df[df['Sentiment']==0].sample(frac=0.99)
df_1 = df[df['Sentiment']==1].sample(frac=0.26)
df_2 = df[df['Sentiment']==2].sample(frac=0.09)
df_3 = df[df['Sentiment']==3].sample(frac=0.21)
df_4 = df[df['Sentiment']==4].sample(frac=0.75)

df = pd.concat([df_0,df_1,df_2,df_3,df_4])
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [6]:
df = df.sample(frac=0.5)

In [7]:
df.shape

(17536, 2)

In [8]:
import numpy as np

seq_len = 512
num_samples = len(df)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

Xids.shape

(17536, 512)

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.4 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(df['Phrase']):
  tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                 add_special_tokens=True,
                                 padding='max_length', return_tensors='tf')

  Xids[i, :] = tokens['input_ids']
  Xmask[i, :] = tokens['attention_mask']

In [ ]:
arr = df['Sentiment'].values

In [ ]:
labels = np.zeros((num_samples, arr.max()+1))

In [ ]:
labels.shape

In [ ]:
labels[np.arange(num_samples), arr] = 1

---

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids' : input_ids, 'attention_mask' : masks}, labels

In [ ]:
dataset = dataset.map(map_func)
dataset.take(1)

In [ ]:
batch_size=16

dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1)

In [ ]:
split = 0.9
size = int((num_samples / batch_size) * split)

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
del dataset

---
**Model**

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-uncased')

bert.summary()

In [ ]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]

x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name='outputs')(x)

In [ ]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, weight_decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy(name='accuracy')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

In [ ]:
model.save('sentiment_model')

---
**Training**

In [ ]:
#model = tf.keras.models.load_model('sentiment_model')

In [ ]:
#def prep_data:

 #   tokens = tokenizer.encode_plus(text, max_length=seq_len, truncation=True,
 #                                  add_special_tokens=True,
 #                                  padding='max_length', return_token_type_id=False,
 #                                  return_tensors='tf')
 #   return {
 #       'input_ids' : tf.cast(tokens['input_ids'], tf.float64)
 #       'attention_mask' : tf.cast(tokens['attention_mask'], tf.float64)
 #   }

In [ ]:
#df_test_0 = prep_data(df_test['Phrase'][0])

In [ ]:
#probs = model.predict(df_test_0)

In [ ]:
#np.argmax(probs[0])